In [ ]:
!pip install tabgan

In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np

train = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv").dropna()
test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv").dropna()

In [ ]:
train.isnull().sum().sum()
test.isnull().sum().sum()

In [ ]:


# random input data

target = pd.DataFrame(train['claim'])
train=train.drop(['claim','id'],axis=1)

test=test.drop('id',axis=1)




# generate data
#new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test, )


new_train3, new_target3 = GANGenerator(gen_x_times=1.1, cat_cols=None, bot_filter_quantile=0.001,
                                       top_filter_quantile=0.999,
                                       is_post_process=True,
                                       adversaial_model_params={
                                           "metrics": "AUC", "max_depth": 2,
                                           "max_bin": 100, "n_estimators": 500,
                                          "learning_rate": 0.02, "random_state": 42,
                                      }, pregeneration_frac=2, only_generated_data=False,
                                      epochs=500).generate_data_pipe(train, target,
                                                                    test, deep_copy=True,
                                                                    only_adversarial=False,
                                                                     use_adversarial=True)

In [ ]:
new_train3.to_csv('train.csv',index=False) 
new_target3.to_csv('targetcsv',index=False) 

# I’m Something of a Painter Myself

**Warm UP**

suppose that a smuggler wants to make fake currency, but dont know how the real currency look. there is a police whos job is to accuratly distinguise fake or real currency.

now the task of smuggler is to generate fake currency as good as real ones.
and the task of police is to correctly identify fake currency.

GAN's story is almost similar. a genral GAN composed of two counterparts 
1. Generator whose job is to create fake data i.e (images).
2. Discriminator whose job is to differenciate fake and real data

In summary for this notebook , the game follows with:
The generator trying to maximize the probability of making the discriminator mistakes its inputs as real.
And the discriminator guiding the generator to produce more realistic images.

In the perfect equilibrium, the generator would capture the general training data distribution. As a result, the discriminator would be always unsure of whether its inputs are real or not.

# Below is the very basic architecture of GAN's

![](http://cdn-media-1.freecodecamp.org/images/m41LtQVUf3uk5IOYlHLpPazxI3pWDwG8VEvU)

# AIM OF THIS NOTEBOOK IS TO DEVELOP BASIC UNDERSTANDING OF GANS BEFORE DIVING INTO COMPETATION

We will be using MNIST dataset, the mnist dataset have image data of numerical digits 0-9
so our gan will be generating fake images of digits. 

**DCGAN on MNIST**
**DCGAN is a Generative Adversarial Network (GAN) using CNN.**

* The discriminator learns to discriminate real from fake images.
* The generator tries to fool the discriminator by generating fake images.
* The generator + discriminator form an adversarial network.
* DCGAN trains the discriminator and adversarial networks alternately.

# PRE-REQUISITES

1. basic inderstanding of neural networks.
2. terminologies like optimaization,upsampling, downsampling etc.
3. little bit of coding related to tensorflow

# IMPORT LIBRARIES

In [ ]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras import layers

# Buiding a Generator Model¶
**Build a model using layers of BatchNorm-ReLU-Conv2DTranpose to generate fake images**

![](https://cdn-media-1.freecodecamp.org/images/cKZw7BBTj3gfDCc8xiSKzf6PyHZkhHEVrFy3)

**The Generator network implemented here**

The network has 4 convolutional layers, all followed by BN (except for the output layer) and Rectified Linear unit (ReLU) activations.

It takes as an input a random vector z (drawn from a normal distribution). After reshaping z to have a 4D shape, we feed it to the generator that starts a series of upsampling layers.

Each upsampling layer represents a transpose convolution operation with strides 2. Transpose convolutions are similar to the regular convolutions.

This final output shape is defined by the size of the training images. In this case for MNIST, it would generate a 28x28 greyscale image.

**WHAT IS UP SAMPLING**
In the Upsampling network, the abstract image representations are upsampled using various techniques to make their spatial dimensions equal to the input image.


In [ ]:
def build_generator(image_size=28, input_size=100):
    
    #Build an input layer
    gen_input = keras.Input(shape=(input_size,))
    
    #Increase dimensions and resize to 3D to feed it to Conv2DTranspose layer
    x = layers.Dense(7 * 7 * 128)(gen_input)
    x = layers.Reshape((7, 7, 128))(x)
    
    #Use ConvTranspose
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(128, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(64, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(32, kernel_size=[5,5], strides=1, padding='same')(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(1, kernel_size=[5,5], strides=1, padding='same')(x)
    
    #Output layer for Generator
    x = layers.Activation('sigmoid')(x)
    
    #Build model using Model API
    generator = keras.Model(gen_input, x, name='generator')
    
    return generator

# Discriminator

please note generator and dicriminator network seems to be same. the difference is that in generator we were using convolution2DTranspose while in discrimator Convolution2D.

for generator we need to increase size from input(100) to target image (28,28)
for discriminator we need to decrease size from 28,28 to 1

The discriminator is also a 4 layer CNN with BN (except its input layer) and leaky ReLU activations. Many activation functions will work fine with this basic GAN architecture.
However, leaky ReLUs are very popular because they help the gradients flow easier through the architecture.

A regular ReLU function works by truncating negative values to 0. This has the effect of blocking the gradients to flow through the network. Instead of the function being zero, leaky ReLUs allow a small negative value to pass through. That is, the function computes the greatest value between the features and a small factor.

![](https://i0.wp.com/androidkt.com/wp-content/uploads/2020/05/Selection_019.png?resize=760%2C364)

Finally, the discriminator needs to output probabilities. For that, we use the Logistic Sigmoid activation function on the final logits.

In [ ]:
def build_discriminator(data_shape=[28,28,1,]):
    
    #Build the network
    dis_input = keras.Input(data_shape)
    x = layers.LeakyReLU(alpha=0.2)(dis_input)
    x = layers.Conv2D(32, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(64, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = tf.keras.layers.Conv2D(256, kernel_size=[5,5], strides=1, padding='same')(x)
    
    #Flatten the output and build an output layer
    x = layers.Flatten()(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    
    #Build Model
    discriminator = keras.Model(dis_input, x, name='discriminator')
    
    return discriminator

# Build Adversarial model

till now we have only created skeleton for our GAN. as of  we need to add compiling part, optimizer, forward pass we need parameters like noise learning rate decay.

**Adversarial = generator + discriminator**

we will be calling build_generator and build_discriminator functions, i am using Adam optimizer which is popular one, experement with using different optimizers
[here is a link to know more abut optimizers](https://machinelearningknowledge.ai/keras-optimizers-explained-with-examples-for-beginners/#:~:text=Keras%20Adam%20Optimizer%20%28Adaptive%20Moment%20Estimation%29%20The%20adam,of%20data%20and%20parameters%20are%20available%20for%20usage.)

**IMPORTANT**
i have not compiled generator, while i have compiled discriminator why because generator will be using some learning of dicriminator. so we will be using frozen disciminator to train the whole adversarial network.

**-->so the ultimate equation be like**
adversarial network = generator + frozen discriminator.



In [ ]:
def build_models():
    
    noise_size = 100
    lr = 2e-4
    decay = 6e-8
    
    #Build Base Discriminator model
    base_discriminator = build_discriminator(data_shape=(28,28,1,))
    
    #Define optimizer and compile model
    discriminator = keras.Model(inputs=base_discriminator.inputs, 
                                          outputs=base_discriminator.outputs)
    optimizer = keras.optimizers.RMSprop(lr=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
    
    #Build Generator model
    generator = build_generator(image_size=28, input_size=noise_size)
    
    #Build Frozen Discriminator
    frozen_discriminator = keras.Model(inputs=base_discriminator.inputs, 
                                          outputs=base_discriminator.outputs)
    #Freeze the weights of discriminator during adversarial training
    frozen_discriminator.trainable = False

    #Build Adversarial model
    optimizer = keras.optimizers.RMSprop(lr=lr * 0.6, decay=decay * 0.5)
    #Adversarial = generator + discriminator
    adversarial = keras.Model(generator.input, 
                        frozen_discriminator(generator.output))
    
    adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])    
    
    return generator, discriminator, adversarial

# NOW COMES THE TRAINING PART

**JUST IN CASE** 
this notebook is not about classification, its about generating fake images from existing ones.

right now load the data and then start training, carefully if you see we have only loaded train part from dataset not the test part and labels. right because we are only interested in creating new images.

first we will be training disciminator followed by training the Adversial network(training generator)  next at evrey iteration we see how well our generator is doing. specifically after 500 iteration we will see the output from generator.

so before begining the training check the architecture of Generator, discriminator and adversarial network and try to analyse how inputs are conveyed in generator network.

discriminator and adversarial network are general purpose neural network.

In [ ]:
def train_gan(generator, discriminator, adversarial, noise_size=100):
    
    #Training parameters
    batch_size = 64
    train_steps = 10000
    image_size = 28
    
    # load MNIST dataset
    (train_x, _), (_, _) = tf.keras.datasets.mnist.load_data()
    #Make it 3D dataset
    train_x = np.reshape(train_x, [-1, image_size, image_size, 1])
    #Standardize data : 0 to 1
    train_x = train_x.astype('float32') / 255
    
    #Input for testing generator at different intervals, we will generate 16 images
    test_noise_input = np.random.uniform(-1.0,1.0, size=[16, noise_size])
    
    #Start training
    for i in range(train_steps):
        
        #Train DISCRIMATOR
        
        #1. Get fake images from Generator
        noise_input = np.random.uniform(-1.0,1.0, size=[batch_size, noise_size])
        fake_images = generator.predict(noise_input)
        
        #2. Get real images from training set
        img_indexes = np.random.randint(0, train_x.shape[0], size=batch_size)
        real_images = train_x[img_indexes]
        
        #3. Prepare input for training Discriminator
        X = np.concatenate((real_images, fake_images))
        
        #4. Labels for training
        y_real = np.ones((batch_size, 1))
        y_fake = np.zeros((batch_size, 1))
        y = np.concatenate((y_real, y_fake))
        
        #5. Train Discriminator
        d_loss, d_acc = discriminator.train_on_batch(X, y)
        
        
        #Train ADVERSARIAL Network
        
        #1. Prepare input - create a new batch of noise
        X = noise_input = np.random.uniform(-1.0,1.0, size=[batch_size, noise_size])
        
        #2. Prepare labels - training Adversarial network to lie :) - All 1s
        y = np.ones((batch_size, 1))
        
        #3. Train - Pls note Discrimator is not getting trained here
        a_loss, a_acc = adversarial.train_on_batch(X, y)
        
        if i % 100 == 0:
            #Print loss and Accuracy for both networks
            print("%s [Discriminator loss: %f, acc: %f, Adversarial loss: %f, acc: %f]" % (i, d_loss, d_acc, a_loss, a_acc) )
        
        #Save generated images to see how well Generator is doing
        if (i+1) % 500 == 0:
            
            #Generate 16 images
            fake_images = generator.predict(test_noise_input)
            
            #Display images
            plot_images(fake_images, i+1)
            
    #Save Generator model
    generator.save('mnist_generator_dcgan.h5')    

# This is a utility function to draw images

In [ ]:
def plot_images(fake_images, step):
    
    plt.figure(figsize=(2.5,2.5))
    num_images = fake_images.shape[0]
    
    image_size = fake_images.shape[1]
    rows = int(math.sqrt(fake_images.shape[0]))
    
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(fake_images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()

# MODEL SUMMARY

In [ ]:
G, D, A = build_models()
G.summary()
D.summary()
A.summary()


# Lets begin the training

**-first iteration-**
initially the generator have no idea, after 500 epochs the generated images doesnt seems like digits.

**-second iteration- Onward** 
the generator have developed little bit understanding of data, geerated images have so similarity to minst dataset

after every cycle it can be visually confirmed that quality of generated images are increasing.

***THANKS FOR PATIENTLY READING TILL END***

In [ ]:
train_gan(G, D, A)